## Import Relevant Packages

In [ ]:
# Display graphs in the jupyter notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Imports here
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Load the Data and Define transforms

In [1]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [2]:
# Done: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Done: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_valid_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=test_valid_transforms)

# Done: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
validationloader = torch.utils.data.DataLoader(validation_data, batch_size=64)

NameError: name 'transforms' is not defined

## Label Mapping

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
print(len(cat_to_name))

## Load Pre-trained Network

In [ ]:
model = models.densenet121(pretrained=True)

# Freze model parameters
for param in model.parameters():
    param.requires_grad = False

## Build Classifier

In [ ]:
model.classifier = nn.Sequential(nn.Linear(1024, 500),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(500, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))

## Implement training pass

In [ ]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

epochs = 8
print_every = 5
steps = 0

for epoch in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Obtain the logarithmic output
        logps = model.forward(inputs)
        
        # Obtain the loss
        loss = criterion(logps, labels)
        
        # Perform backward propagation
        loss.backward()
        
        # Take an optimizer step
        optimizer.step()

        # Update the loss so far.
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Initialize variables to keep track of training loss and testing loss
            test_loss = 0
            accuracy = 0
            
            # Set model to evaluation mode.
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

## Testing network with validation set

In [ ]:
# Done: Do validation on the test set
model.eval()
accuracy = 0
test_loss = 0
with torch.no_grad():
    for inputs, labels in validationloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(testloader):.3f}.. "
      f"Test accuracy: {accuracy/len(testloader):.3f}")

## Saving the trained model

In [ ]:
# Done: Save the checkpoint
checkpoint = {'input_size': 1024,
              'output_size': 102,
              'hidden_layers': [500, 256],
              'state_dict': model.state_dict()}
torch.save(checkpoint, 'checkpoint.pth')

## Define function to lead from checkpoint

In [ ]:
# Done: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    # Load pre-trained model
    model = models.densenet121(pretrained=True)

    # Freze model parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Build new classifier to fit model
    model.classifier = nn.Sequential(nn.Linear(checkpoint['input_size'], checkpoint['hidden_layers'][0]),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(checkpoint['hidden_layers'][0], checkpoint['hidden_layers'][1]),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(checkpoint['hidden_layers'][1], checkpoint['output_size']),
                                 nn.LogSoftmax(dim=1))
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

## Inference from input image

In [ ]:
from PIL import Image

def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Load the image
    im = Image.open(image_path)
    
    # Resize image
    currWidth, currHeight = im.size
    
    if currWidth < currHeight:
        newHeight = int(currHeight*256/currWidth)
        im.resize((256, newHeight))
    else:
        newWidth = int(currWidth*256/currHeight)
        im.resize((newWidth, 256))
    
    # Crop the image
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    im = im.crop((left, top, right, bottom))
    
    # Convert image into numpy array and normalize
    np_image = np.array(im)/255
    np_image  = (np_image - np.array([0.485, 0.456, 0.406]))/np.array([0.229, 0.224, 0.225])
    
    # Tanspose the image
    np_image = np.transpose(np_image, (2, 0, 1))
    
    # Convert image to a tensor.
    image_tensor = torch.from_numpy(np_image)
    
    return image_tensor

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # Obtain image and model using functions defined above
    img_tensor = process_image(image_path)
    model = load_checkpoint(model)
    
    # Pass image through the model
    logps = model.forward(inputs)

    # Get prediction
    ps = torch.exp(logps)
    top_p, top_class = ps.topk(topk, dim=1)
    
    # Debug output
    print(top_p)
    print(top_class)
    
    return top_p, top_class